**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

software = '/home/mario/DockM8/software'
protein_file = '/home/mario/DockM8/cdk2/receptor_protoss_prepared_pocket.pdb'
ref_file = '/home/mario/DockM8/cdk2/crystal_ligand_protoss.sdf'
docking_library = '/home/mario/DockM8/cdk2/merged_actives_decoys.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'RTMScore', 'SCORCH', 'vinardo']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'pkasolver'
parallel = 1
ncpus = int(os.cpu_count()-2)
pocket = 'reference'
#Create a temporary folder for all further calculations
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

[10:42:39] Initializing Normalizer
/home/mario/anaconda3/envs/wocondock/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.


The working directory has been set to: /home/mario/DockM8/cdk2
The folder: /home/mario/DockM8/cdk2/temp already exists


/home/mario/anaconda3/envs/wocondock/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.netcdf.netcdf_file):


In [ ]:
if os.path.isfile(protein_file.replace('.pdb', '_pocket.pdb')) == False:
    if pocket == 'reference':
        pocket_definition = GetPocket(ref_file, protein_file, 8)
    elif pocket == 'dogsitescorer':
        pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

In [ ]:
if os.path.isfile(w_dir+'/temp/final_library.sdf') == False:
    prepare_library(docking_library, id_column, software, protonation, ncpus)

In [ ]:
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
docking(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses, ncpus)


In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')


In [ ]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir+f'/temp/clustering/{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

In [ ]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, 20)


In [ ]:
calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)

In [3]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', ['gnina', 'RTMScore'])

The folder: /home/mario/DockM8/cdk2/temp/ranking already exists
['Pose ID', 'GNINA', 'CNN-Score', 'CNN-Affinity', 'RTMScore']
                     Pose ID     GNINA  CNN-Score  CNN-Affinity  RTMScore
0       ZINC26203125_GNINA_1  0.953189   0.701066      0.661387  0.338430
1       ZINC16042498_GNINA_1  0.954882   0.509752      0.642799  0.229693
2       ZINC12034275_GNINA_1  0.953792   0.869457      0.599097  0.307842
3       ZINC20465253_GNINA_1  0.954682   0.771392      0.625932  0.291665
4       ZINC27735434_GNINA_1  0.951399   0.916446      0.733753  0.315982
...                      ...       ...        ...           ...       ...
3519  ZINC23154170_PLANTS_01  0.951301   0.119107      0.581674  0.094072
3520  ZINC30000959_PLANTS_01  0.952716   0.223677      0.701696  0.239031
3521  ZINC25166358_PLANTS_01  0.915143   0.236323      0.403006  0.262473
3522  ZINC17363296_PLANTS_01  0.962456   0.494097      0.734661  0.330200
3523  ZINC32096851_PLANTS_01  0.952216   0.591271      0.518